In [366]:
import requests
import pandas as pd
import json

# Pull Latitude and Longitude from positionstack.com based on geographical user input.

In [367]:
# User input: Country
country_input = (input("Enter a Country using a 2-letter country code (Example: US): "))

Enter a Country using a 2-letter country code (Example: US): jp


In [368]:
# N/A: No need for this extra geographic variable.

# User input: State
# region_input = (input("Enter a region (City, County, State, etc.): "))

In [369]:
# User input: City

# HTTP GET Request Parameters:
# query	[Required] Specify your query as a free-text address, 
# place name or using any other common text-based location 
# identifier (e.g. postal code, city name, region name).

query_input = (input("Enter a location (City, Zip Code, etc.): "))

Enter a location (City, Zip Code, etc.): tokyo


In [370]:
# Geo reference for adding to final dataframe.
geo_input = country_input  + " | " + query_input
geo_input

'jp | tokyo'

In [371]:
import http.client, urllib.parse

conn = http.client.HTTPConnection('api.positionstack.com')

params = urllib.parse.urlencode({
    'access_key': '1fbf7de4ce05274e9b9005659970c429',
    
    # Variables get live user input above.
    'country': country_input,
#     'region': region_input,
    'query': query_input,
        
    # "limit" of 1 ensures only 1 Lat/Lon will be passed below.    
    'limit': 1
    })

conn.request('GET', '/v1/forward?{}'.format(params))

res = conn.getresponse()
geo = res.read()
geo_str = (geo.decode('utf-8'))

print(geo.decode('utf-8'))

###########################################################
# Still needs error handling.

{"data":[{"latitude":35.695126,"longitude":139.75391,"type":"locality","name":"Tokyo","number":null,"postal_code":null,"street":null,"confidence":1,"region":"Tokyo","region_code":"TK","county":"Chiyoda","locality":"Tokyo","administrative_area":null,"neighbourhood":null,"country":"Japan","country_code":"JPN","continent":"Asia","label":"Tokyo, Japan"}]}


In [372]:
# Help from AskBCS:
# Result was one long string, that only looked 
# like a dictionary, but wasn't.
# Use "type()" to identify whether or not 
# result is actually a dict.

type(geo_str)

str

In [373]:
# Convert string to json (which is a dictionary).
# geo_json
geo_json = json.loads(geo_str)

print(geo_json)

{'data': [{'latitude': 35.695126, 'longitude': 139.75391, 'type': 'locality', 'name': 'Tokyo', 'number': None, 'postal_code': None, 'street': None, 'confidence': 1, 'region': 'Tokyo', 'region_code': 'TK', 'county': 'Chiyoda', 'locality': 'Tokyo', 'administrative_area': None, 'neighbourhood': None, 'country': 'Japan', 'country_code': 'JPN', 'continent': 'Asia', 'label': 'Tokyo, Japan'}]}


##### Reminder: Double parentheses are functional in the for loops.

In [374]:
# # geo_json_coord = lat and lon combined into a tuple.

# geo_json_coord = geo_json['data']
# latlon_tup = []
# for x in geo_json_coord:
#     latlon_tup.append((x['latitude'], x['longitude']))

In [375]:
# Pull latitude value by itself.

geo_json_lat = geo_json['data']
var_lat = []
for x in geo_json_lat:
    var_lat.append((x['latitude']))
    
var_lat[0]

35.695126

In [376]:
# Pull longitude value by itself.

geo_json_lon = geo_json['data']
var_lon = []
for x in geo_json_lon:
    var_lon.append((x['longitude']))
    
var_lon[0]

139.75391

# Push Latitude and Longitude to Weather API URL, based on start date and stop date from user input.

In [377]:
# List of columns, for starter dataset:

# hourly_columns = ['time',
#              'temperature_2m_degF', 
#              'relativehumidity_2m_perc', 
#              'rain_inch',
#              'snowfall_cm', 
#              'cloudcover_perc', 
#              'windspeed_10m_mph',
#              'winddirection_10m_deg']

#  We pulled data from 2000-01-01 to 2020-12-31.

Original Open Meteo API URL:

https://archive-api.open-meteo.com/v1/era5?latitude=52.52&longitude=13.41&start_date=2000-01-01&end_date=2020-12-31&hourly=temperature_2m,relativehumidity_2m,rain,snowfall,cloudcover,windspeed_10m,winddirection_10m&timezone=America%2FLos_Angeles&temperature_unit=fahrenheit&windspeed_unit=mph&precipitation_unit=inch

In [379]:
# User input: Start Date
start_date_input = (input("Enter the start date (YYYY-MM-DD): "))

Enter the start date (YYYY-MM-DD): 2010-01-01


In [380]:
# User input: End Date
end_date_input = (input("Enter the end date (YYYY-MM-DD): "))

Enter the end date (YYYY-MM-DD): 2020-12-31


In [381]:
# Break URL into snippets to be assembled below:

url_snip_1 = "https://archive-api.open-meteo.com/v1/era5?latitude=" 
url_snip_2 = str(var_lat[0])   # latitude, input from previous json
url_snip_3 = "&longitude="
url_snip_4 = str(var_lon[0])   # longitude, input from previous json



url_snip_5 = "&start_date="
url_snip_6 = start_date_input   # start date, input from user
# url_snip_6 = "2010-01-01"   # --- OR --- static start date, for dummy data

url_snip_7 = "&end_date="
url_snip_8 = end_date_input   # end date, input from user
# url_snip_8 = "2020-12-31"   # --- OR ---  static end date, for dummy data



url_snip_9 = "&hourly="   # elements, hourly
url_snip_10 = "temperature_2m,"   # data element
url_snip_11 = "relativehumidity_2m,"   # data element
url_snip_12 = "rain,snowfall,"   # data element
url_snip_13 = "cloudcover,"   # data element
url_snip_14 = "windspeed_10m,"   # data element
url_snip_15 = "winddirection_10m"   # data element

url_snip_16 = "&timezone=America%2FLos_Angeles"   # time zone
url_snip_17 = "&temperature_unit=fahrenheit"   # temp unit
url_snip_18 = "&windspeed_unit=mph"   # windspeed unit
url_snip_19 = "&precipitation_unit=inch"   # precip unit

In [382]:
# Contatenate the url snippets.

url_concat_full = \
    url_snip_1 + \
    url_snip_2 + \
    url_snip_3 + \
    url_snip_4 + \
    url_snip_5 + \
    url_snip_6 + \
    url_snip_7 + \
    url_snip_8 + \
    url_snip_9 + \
    url_snip_10 + \
    url_snip_11 + \
    url_snip_12 + \
    url_snip_13 + \
    url_snip_14 + \
    url_snip_15 + \
    url_snip_16 + \
    url_snip_17 + \
    url_snip_18 + \
    url_snip_19

In [383]:
print(url_concat_full)

https://archive-api.open-meteo.com/v1/era5?latitude=35.695126&longitude=139.75391&start_date=2010-01-01&end_date=2020-12-31&hourly=temperature_2m,relativehumidity_2m,rain,snowfall,cloudcover,windspeed_10m,winddirection_10m&timezone=America%2FLos_Angeles&temperature_unit=fahrenheit&windspeed_unit=mph&precipitation_unit=inch


In [384]:
# Data comes in as one long string.
weather_str = requests.get(url_concat_full).text

In [385]:
# Convert string to dictionary.
weather_json = json.loads(weather_str)

In [386]:
# Verify that data type is dictionary.
type(weather_json)

dict

In [387]:
# Convert json dictionary into dataframe.

# Standard "pd.json_normalize(weather_json)" doesn't work 
# with these nested levels. Also tried adding "record_path"
# and "meta" tags, but still didn't work.
# Found "from_records" alternative that does work.

# N/A: Code from:
# https://stackoverflow.com/questions/66894920/json-normalise-from-nested-api-to-df-throwing-type-error
# Creates "pivot" table format with all entries as columns, 
# all variables as rows.
# Not what we need, but might be useful for other datasets.
# dftest3 = pd.DataFrame.from_records(weather_json['hourly']).T.reset_index()


# Code from:
# https://stackoverflow.com/questions/66894920/json-normalise-from-nested-api-to-df-throwing-type-error

weather_raw = pd.DataFrame.from_records(weather_json['hourly'])
weather_raw

,cloudcover,rain,relativehumidity_2m,snowfall,temperature_2m,time,winddirection_10m,windspeed_10m
0,1,0.0,37,0.0,45.0,2010-01-01T00:00,15.0,8.6
1,0,0.0,48,0.0,41.5,2010-01-01T01:00,21.0,8.1
2,0,0.0,54,0.0,39.3,2010-01-01T02:00,16.0,7.4
3,4,0.0,62,0.0,35.8,2010-01-01T03:00,24.0,3.9
4,1,0.0,64,0.0,34.9,2010-01-01T04:00,20.0,5.2
...,...,...,...,...,...,...,...,...
96427,38,0.0,42,0.0,44.3,2020-12-31T19:00,4.0,3.4
96428,26,0.0,38,0.0,46.7,2020-12-31T20:00,27.0,4.0
96429,8,0.0,37,0.0,47.6,2020-12-31T21:00,53.0,4.8
96430,18,0.0,35,0.0,48.0,2020-12-31T22:00,62.0,5.3


In [388]:
# Add geo input from user as new column, for reference.

weather_raw['geo'] = geo_input
weather_raw

,cloudcover,rain,relativehumidity_2m,snowfall,temperature_2m,time,winddirection_10m,windspeed_10m,geo
0,1,0.0,37,0.0,45.0,2010-01-01T00:00,15.0,8.6,jp | tokyo
1,0,0.0,48,0.0,41.5,2010-01-01T01:00,21.0,8.1,jp | tokyo
2,0,0.0,54,0.0,39.3,2010-01-01T02:00,16.0,7.4,jp | tokyo
3,4,0.0,62,0.0,35.8,2010-01-01T03:00,24.0,3.9,jp | tokyo
4,1,0.0,64,0.0,34.9,2010-01-01T04:00,20.0,5.2,jp | tokyo
...,...,...,...,...,...,...,...,...,...
96427,38,0.0,42,0.0,44.3,2020-12-31T19:00,4.0,3.4,jp | tokyo
96428,26,0.0,38,0.0,46.7,2020-12-31T20:00,27.0,4.0,jp | tokyo
96429,8,0.0,37,0.0,47.6,2020-12-31T21:00,53.0,4.8,jp | tokyo
96430,18,0.0,35,0.0,48.0,2020-12-31T22:00,62.0,5.3,jp | tokyo


In [389]:
# San Francisco
# New York
# London
# Berlin
# Tokyo

# sf_df
# ny_df
# london_df
# berin_df
# tokyo_df

# Start Date: 2010-01-01
# End Date: 2020-12-31

# tokyo_df = weather_raw

# tokyo_df

In [390]:
# dummy5cities = pd.concat([sf_df, ny_df, london_df, berin_df, tokyo_df])

In [391]:
# dummy5cities.to_csv('dummy5cities.csv')